In [1]:
from pyomo.environ import *
import math 


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
#this function prints the results of the solver status of the program
def solverstatus(results):
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        print('feasible')
    elif (results.solver.termination_condition == TerminationCondition.infeasible):
        print('infeasible')
    else:
        print ('Solver Status:',  results.solver.status)
#this function adds 2 lists together and removes the identical elements
def remove_dup(first_list,second_list):
    return(list(dict.fromkeys(first_list + second_list)))   

    
model = AbstractModel()
model.i = Set(doc='set of Gen units')
model.b = Param(model.i, doc='b coef of cost')
model.pmin = Param(model.i, doc='Min gen')
model.pmax = Param(model.i, doc='Max gen')


model.GBconnect = Set(dimen=2, doc='Generator connected to bus')

model.lines = Set(dimen=3,doc = "Lines connecting bus and node")
model.branchX = Param(model.lines, doc='Max gen')
model.branchLim = Param(model.lines, doc='Max gen')

model.bus = Set()
model.node = Set(initialize=model.bus)

model.sbase = Param(doc='Sbase')
model.Pd = Param(model.bus, doc='Max gen')
model.delta = Var(model.bus, bounds=(-math.pi, math.pi), domain=Reals, doc='angle')




#Defines the Pmin and Pmax of the busses
def Pbounds(model,i):
     return (model.pmin[i]/model.sbase, model.pmax[i]/model.sbase)
model.P = Var(model.i, bounds=Pbounds, domain=Reals)

#Objective Function for Bus and nodes.
def objFunc(model):
    return sum(model.b[i]*model.P[i] for i in model.i)
model.objective_function = Objective(rule=objFunc, sense = minimize)


# Problem iterating over the buses, it only goes over one direction, i.e flow from 1 to 2 and ignores 2 to 1.
# Because the flow is iterated over model.lines only so the only method is to either add theese lines explicitly
# in the data file. 
def LineBounds(model,lines,bus,node):
    
    if (lines,bus,node)  in model.lines:
        #print(lines,bus,node)
        print("Flag 1")
        return(-model.branchLim[lines,bus,node]/model.sbase,model.branchLim[lines,bus,node]/model.sbase)
    if (lines,node,bus)  in model.lines:
        print("Flag 2")
        return(-model.branchLim[lines,node,bus]/model.sbase,model.branchLim[lines,node,bus]/model.sbase)

    else:
        return (0,0)    
model.flow = Var( model.lines,bounds = LineBounds , domain=Reals, doc='flow')



#this function specifies which bus is attached to which nodes. [correct]
def NodesOut_init(model, bus):
    
    retval1 = []
    retval2 = []
    retval = []
    for (l,i,j)  in model.lines:
        if i == bus and j != bus:
            #print(i,j,bus)
            retval1.append(j)
    for (l,j,i)  in model.lines:
        if i == bus and j != bus:
            retval2.append(j)
    retval.append(remove_dup(retval1,retval2))
    #print(str(retval))
    return retval
model.NodesOut = Set(model.bus, initialize=NodesOut_init)


#this function will specify which generator is attached to what bus. [correct]
def Nodeswithgen(model, bus):
    retval = []
    for (gen,node) in model.GBconnect:
        if node == bus:
            retval.append(gen)
    #print(retval) 
    return retval
model.GB = Set(model.bus, initialize=Nodeswithgen)

#Definies the flow on each line, this function has the same issue of "LineBounds" function where it works but only for the buses and nodes provided. 
def linecalc_rule(model,line,bus,node):
    if (line,bus,node) in model.lines:
        #print("y")
        return model.flow[line,bus,node] == (1/model.branchX[line,bus,node])*(model.delta[bus] - model.delta[node])
    elif (line,node,bus) in model.lines:
        #print("x")
        return model.flow[line,node,bus] == (1/model.branchX[line,node,bus])*(model.delta[bus] - model.delta[node])    
model.constflowcalc = Constraint(model.lines, rule=linecalc_rule)


#defines the balance rule on each bus 
def balance_rule(model,bus):
    for i in model.GB[bus]:
        print(model.P[i],model.GB[bus])
    return sum(model.P[i] for i in model.GB[bus]) - model.Pd[bus]/model.sbase == sum(model.flow[line,b,node] for (line,b,node) in model.lines if b == bus)
    return (0,0)

model.constbalance = Constraint(model.bus , rule=balance_rule)


opt = SolverFactory('glpk')
instance = model.create_instance("data3.dat")
results = opt.solve(instance)


Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
Flag 1
P[g1] GB[b1]
P[g2] GB[b1]
P[g3] GB[b3]
P[g4] GB[b4]
P[g5] GB[b5]


In [4]:
solverstatus(results)

feasible


In [5]:
instance.pprint()

7 Set Declarations
    GB : Size=5, Index=bus, Ordered=Insertion
        Key : Dimen : Domain : Size : Members
         b1 :     1 :    Any :    2 : {'g1', 'g2'}
         b2 :    -- :    Any :    0 : {}
         b3 :     1 :    Any :    1 : {'g3',}
         b4 :     1 :    Any :    1 : {'g4',}
         b5 :     1 :    Any :    1 : {'g5',}
    GBconnect : Generator connected to bus
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :    5 : {('g1', 'b1'), ('g2', 'b1'), ('g3', 'b3'), ('g4', 'b4'), ('g5', 'b5')}
    NodesOut : Size=5, Index=bus, Ordered=Insertion
        Key : Dimen : Domain : Size : Members
         b1 :     3 :    Any :    1 : {('b2', 'b4', 'b5'),}
         b2 :     2 :    Any :    1 : {('b3', 'b1'),}
         b3 :     2 :    Any :    1 : {('b4', 'b2'),}
         b4 :     3 :    Any :    1 : {('b5', 'b1', 'b3'),}
         b5 :     2 :    Any :    1 : {('b1', 'b4'),}
    bus : Size=1, Index=None, Ord

In [6]:
print("---System Data---")
print("--Power Demand at each bus--")
for pd in instance.bus:
     print(pd , instance.Pd[pd])
print("--Branch Limits--")
for bl in instance.lines:
     print(bl , instance.branchLim[bl])
print("--Line Impeadance--")
for imp in instance.lines:
     print(imp , instance.branchX[imp])

---System Data---
--Power Demand at each bus--
b1 0
b2 300
b3 300
b4 400
b5 0
--Branch Limits--
('L1', 'b1', 'b2') 400
('L2', 'b1', 'b4') 400
('L3', 'b1', 'b5') 400
('L4', 'b2', 'b3') 400
('L5', 'b3', 'b4') 400
('L6', 'b4', 'b5') 240
('L1', 'b2', 'b1') 400
('L2', 'b4', 'b1') 400
('L3', 'b5', 'b1') 400
('L4', 'b3', 'b2') 400
('L5', 'b4', 'b3') 400
('L6', 'b5', 'b4') 240
('L7', 'b4', 'b5') 240
('L7', 'b5', 'b4') 240
--Line Impeadance--
('L1', 'b1', 'b2') 0.0281
('L2', 'b1', 'b4') 0.0304
('L3', 'b1', 'b5') 0.0064
('L4', 'b2', 'b3') 0.0108
('L5', 'b3', 'b4') 0.0297
('L6', 'b4', 'b5') 0.0297
('L1', 'b2', 'b1') 0.0281
('L2', 'b4', 'b1') 0.0304
('L3', 'b5', 'b1') 0.0064
('L4', 'b3', 'b2') 0.0108
('L5', 'b4', 'b3') 0.0297
('L6', 'b5', 'b4') 0.0297
('L7', 'b4', 'b5') 0.0297
('L7', 'b5', 'b4') 0.0297


In [7]:
print("---Results---")
print(value(instance.objective_function.expr)*instance.sbase)

print("--Flow--")
for flow in instance.flow:
    if type(instance.flow[flow].value) is float and instance.flow[flow].value >= 0:
        print( flow, round(instance.flow[flow].value,5)*instance.sbase)
            
print("--delta--")
for b in instance.bus:
        print(b , instance.delta[b].value)
print("--generation--")
for i in instance.i:
     print(i , round(instance.P[i].value*instance.sbase,5))

---Results---
14809.99999999976
--Flow--
('L1', 'b1', 'b2') 287.123
('L2', 'b1', 'b4') 140.777
('L3', 'b5', 'b1') 217.89999999999998
('L4', 'b3', 'b2') 12.876999999999999
('L5', 'b4', 'b3') 122.877
('L6', 'b5', 'b4') 191.05
('L7', 'b5', 'b4') 191.05
--delta--
b1 -3.06091114092267
b2 -3.14159265358979
b3 -3.14020191824477
b4 -3.10370739604597
b5 -3.04696554246568
--generation--
g1 40.0
g2 170.0
g3 190.0
g4 0.0
g5 600.0
